In [1]:
import numpy as np

In [2]:
# creating the environment
number_of_states = 16
number_of_actions = 4 

In [3]:
next_state = np.zeros((16,4),np.int)
terminal_state = np.zeros((16,4))
reward = np.zeros((16,4))

# creating a transition probability matrix next_state[s][a]which stores the next state it goes to after completing the action,
#reward[s][a] stores the reward  it gets
# and terminal_state[s][a] whether it reaches the terminal step or not
for i in range(4):
    next_state[0][i]= 0
    reward[0][i]=0
    terminal_state[0][i]=True
for i in range(4):
    next_state[15][i]= 15
    reward[15][i]=0
    terminal_state[15][i]=True
for i in range(1,15):
    for j in range(4):
        reward[i][j] = -1
for i in range(1,15):
    if(i-4>=0):
        next_state[i][0]= i-4
    else:
        next_state[i][0] = i
    if(next_state[i][0]==0 or next_state[i][0]==15):
        terminal_state[i][0]= True
        
    else:
        terminal_state[i][0]=False
for i in range(1,15):
    if(i%4!=0):
        next_state[i][1]= i-1
    else:
        next_state[i][1] = i
    if(next_state[i][1]==0 or next_state[i][1]==15):
        terminal_state[i][1]= True
    else:
        terminal_state[i][1]=False
for i in range(1,15):
    if(i%4!=3):
        next_state[i][2]= i+1
    else:
        next_state[i][2] = i
    if(next_state[i][2]==0 or next_state[i][2]==15):
        terminal_state[i][2]= True
    else:
        terminal_state[i][2]=False
for i in range(1,15):
    if(i+4<=15):
        next_state[i][3]= i+4
    else:
        next_state[i][3] = i
    if(next_state[i][3]==0 or next_state[i][3]==15):
        terminal_state[i][3]= True
    else:
        terminal_state[i][3]=False
rewards = np.zeros((16,1))
for i in range(16):
    if(i==0 or i==15):
        rewards[i] = 0
    else :
        rewards[i]= -1
# creating the environment completed      

### policy evaluation

In [4]:
# policy evaluation
def policy_evaluation(policy, next_state, reward ):
    # value function
    discount_factor=0.999
    theta = 0.00000001
    V = np.zeros(16)
    while True:
        delta = 0
        #finding value function for every state
        for s in range(int(16)):
            v = 0
            # Look at the possible next actions
            for i in range(int(4)):
                # For each action, look at the possible next states
                 v += policy[s][i] *1* (reward[s][i]+ discount_factor * V[next_state[s][i]])
            # seeing how much our value function changed 
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)


In [5]:
# creating a random policy for every action of each state with equal probability
policy = np.ones([16,4])/4
v = policy_evaluation(policy , next_state, reward)


In [6]:
#printing the final value function
print("value function")
print(v.reshape(4,4))

value function
[[  0.         -13.76222671 -19.64826244 -21.60700716]
 [-13.76222671 -17.68951773 -19.65022119 -19.64826245]
 [-19.64826244 -19.65022119 -17.68951773 -13.76222672]
 [-21.60700716 -19.64826245 -13.76222672   0.        ]]


# policy iteration

In [7]:
#implementing policy iteration
def policy_iteration(next_state, reward):
    discount_factor = 0.99
    # finding the q values of every action corresponding to a given state 
    def q_value(state, V):
        A = np.zeros([4,1])
     
        for a in range(4):
            A[a] = 1* (reward[state][a] + discount_factor* V[next_state[state][a]])
        return A
     # making  a random policy
    policy = np.ones([16,4])/4
    while True:
        # evaluating the current policy by our previous function
        V = policy_evaluation(policy, next_state , reward)
       
        optimal_policy = True
        for s in range(16):
            # finding which action will be taken according to the current policy
            chosen_action = np.argmax(policy[s])
            # finding q values corresponding to each action in the given state
            q_values = q_value(s,V)
            best_action = np.argmax(q_values)
            
            # making the policy greedy by taking the best action
            if(best_action!=chosen_action):
                optimal_policy = False
            policy[s] = np.eye(1,4, best_action)
            
        # if we have found the optimal policy, stop iterating    
        if(optimal_policy):
            return policy,V
          
            
            
    

In [8]:
policy, v = policy_iteration(next_state, reward)
print("value function")
print(v.reshape(4,4))

print("policy function for each state where 0-> go up 1-> go left 2-> go right 3-> go down ")
print(np.reshape(np.argmax(policy, axis=1), (4,4)))


value function
[[ 0.       -1.       -1.999    -2.997001]
 [-1.       -1.999    -2.997001 -1.999   ]
 [-1.999    -2.997001 -1.999    -1.      ]
 [-2.997001 -1.999    -1.        0.      ]]
policy function for each state where 0-> go up 1-> go left 2-> go right 3-> go down 
[[0 1 1 1]
 [0 0 0 3]
 [0 0 2 3]
 [0 2 2 0]]


# value iteration

In [9]:
def value_iteration(next_state, reward):
    theta = 0.0001
    discount_factor = 0.99
    # initialising value function to zero
    
    V = np.zeros((16,1))

    while True:
            # usd for terminating condition
            delta = 0
            # going through all the states
            for s in range(16):
                # finding q_value for each action in the state
                A = np.zeros((4,1))
                for a in range(4):
                    A[a] = np.max(reward[s][a] + discount_factor * V[next_state[s][a]])
                # choosing the best action
                best_action = np.max(A)
                # finding how much the value function changes
                delta = max(delta, np.abs(best_action - V[s]))
                # updating the value function for that state
                V[s] = best_action
            # checking if the value function has started converging
            if(delta<theta):
                 break
    # finding the optimal policy for each state
    policy = np.zeros((16,4))
    for s in range(16):
        A = np.zeros((4,1))
        for a in range(4):
            A[a]= reward[s][a] + discount_factor*V[next_state[s][a]]
        # choosing the action which has  the highest q_value
        best_action = np.argmax(A)
        # always selecting that action for that state
        policy[s][best_action] = 1
    return policy , V         

In [10]:
policy , v = value_iteration(next_state, reward)

In [11]:
print("value function")
print(v.reshape(4,4))
print("policy function for each state where 0-> go up 1-> go left 2-> go right 3-> go down ")
print(np.reshape(np.argmax(policy, axis=1), (4,4)))

value function
[[ 0.     -1.     -1.99   -2.9701]
 [-1.     -1.99   -2.9701 -1.99  ]
 [-1.99   -2.9701 -1.99   -1.    ]
 [-2.9701 -1.99   -1.      0.    ]]
policy function for each state where 0-> go up 1-> go left 2-> go right 3-> go down 
[[0 1 1 1]
 [0 0 0 3]
 [0 0 2 3]
 [0 2 2 0]]
